In [1]:
# pip3 install mariadb
# pip3 install pandas
# pip3 install numpy  
# sudo apt-get install libatlas-base-dev
# pip3 install matplotlib
# pip3 install ipympl
# pip3 install ipywidgets

%matplotlib widget
%config InlineBackend.figure_format = 'png'

import os
import sys
import mariadb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, interactive_output, fixed, interact_manual, IntSlider, IntRangeSlider, HBox
import ipywidgets as widgets

In [16]:
class _ss_dict:
    def __init__(self, span: tuple, avg_value: float):
        self.span=span
        self.avg_value=avg_value
    def gen_series(self, leng: int,): # series of avg_value @ certain time span
        _series = pd.Series(0, index=range(leng))
        for i in range(self.span[0], self.span[-1]):
            _series[i] = self.avg_value
        self.series = _series
    def __repr__(self):
        rep = f'Steady State@{self.span}, avg: {self.avg_value}'
        return rep

class SerEnergy_Set_Point:
    density_H2 = 0.08988 # kg/m3 @ STP
    enthalpy_H2 = 120.21 # MJ/kg-LHV
    enthalpy_MeOH = 20.09 # MJ/kg-LHV
    MeOH_weight = 0.543 # 54.3wt%
    cont_step = 60 #continue for 1 min
    TC_range = 5 #5oC
    Scale_range = 5 #5g/min
    continuity = 10 #10min
    def __init__(self, name: str, temp: float, weight_rate: float, dummy: float):
        self.name = name
        # set points
        self.temp = temp 
        self.weight_rate = weight_rate
        self.dummy = dummy
        
    def cond(self, df: pd.core.frame.DataFrame, TC: str, Scale: str):
        self.ss_time = []
        count_60 = 0 #counter for steady mins
        ss_temp = []
        ss_lst = []
        ss = ((df[TC] >= (self.temp - self.TC_range)) & (df[TC] <= (self.temp + self.TC_range))) \
            & ((df[Scale] >= (self.weight_rate - self.Scale_range)) & (df[Scale] <= (self.weight_rate + self.Scale_range)))
        #print(ss.sum())
        for i in range(0, len(ss), self.cont_step):
            if i + self.cont_step <= len(ss):
                if ss.iloc[i:i + self.cont_step].sum() == 60:
                    count_60 += 1
                    ss_temp.append(i)
                else:
                    count_60 = 0
                    if ss_temp != []:
                        ss_lst.append(ss_temp)
                        ss_temp = []
                    continue
        #print(ss_lst)
        for u in ss_lst:
            if len(u) >= self.continuity:
                self.ss_time.append((u[0], u[-1]))
        self.ss_time = tuple(self.ss_time)
    def eff_calc(self, df: pd.core.frame.DataFrame, Scale: str, DFM_RichGas: str, GA_H2: str):
        if self.ss_time:
            self.ss_weight_rate = []
            self.ss_DFM_RichGas = []
            self.ss_GA_H2 = []
            self.heff = []
            for u in self.ss_time:
                #print(u)
                ls = [v for v in range(u[0],u[-1])]
                df_mean = df[df.index.isin(ls)]
                _avg_Scale = df_mean[Scale].mean()
                _avg_DFM_RichGas = df_mean[DFM_RichGas].mean()
                _avg_GA_H2 = df_mean[GA_H2].mean()
                self.ss_weight_rate.append(_ss_dict(span=u, avg_value=_avg_Scale))
                self.ss_DFM_RichGas.append(_ss_dict(span=u, avg_value=_avg_DFM_RichGas))
                self.ss_GA_H2.append(_ss_dict(span=u, avg_value=_avg_GA_H2))
                _avg_H2_flow = self.density_H2 * ((_avg_DFM_RichGas * _avg_GA_H2) / 100 - self.dummy) / 1000 # kg/min @ STP
                _heff = ((_avg_H2_flow * self.enthalpy_H2) / (_avg_Scale / 1000 * self.MeOH_weight * self.enthalpy_MeOH)) * 100 # enthalpy eff [%]
                self.heff.append(_ss_dict(span=u, avg_value=_heff))
            self.ss_weight_rate = tuple(self.ss_weight_rate)
            self.ss_DFM_RichGas = tuple(self.ss_DFM_RichGas)
            self.ss_GA_H2 = tuple(self.ss_GA_H2)
            self.heff = tuple(self.heff)
            #print(self.name, self.ss_weight_rate, self.ss_DFM_RichGas, self.ss_GA_H2)
            #print(self.name, self.heff)
        else:
            pass
    def gen_dataframe(self): # dataframe of summaries
        if self.ss_time:
            _sum_rows = []
            for i in range(len(self.ss_time)):
                _sum_rows.append(pd.DataFrame([[self.name, self.ss_time[i][0], self.ss_time[i][-1], round(self.ss_weight_rate[i].avg_value, 2), round(self.ss_DFM_RichGas[i].avg_value, 2), round(self.ss_GA_H2[i].avg_value, 2), round(self.heff[i].avg_value, 2)]], 
                                                 columns=['Steady State', 'Init[s]', 'End[s]', 'Avg_WeightRate[g/min]', 'Avg_RichGas[LPM]', 'Avg_H2_Conc[%]', 'Enthalpy_Eff[%]'])
                                   )
            self.sum_rows = pd.concat(_sum_rows)
        else:
            pass
        

In [17]:
def plot(Table_name: str, Time: tuple, SS: bool, Calc: bool, Ploting: bool):
    if Calc or Ploting:
        #print(Table_name, Time)
        cur.execute(f"SELECT * FROM {Table_name}")
        df = pd.DataFrame(cur.fetchall(), columns=[entry[0] for entry in cur.description]).set_index('Id')
        _leng = len(df)
        #display(df.head())

        _bls=[]
        _heffls = 0
        for i in Set_Point_lst:
            i.cond(df, TC='TC10', Scale='Scale')
            i.eff_calc(df, Scale='Scale', DFM_RichGas='DFM_RichGas', GA_H2='GA_H2')
            if i.ss_time:
                for u in i.ss_time:
                    #print(u)
                    for v in range(u[0], u[-1]):
                        _bls.append(v)
                for q in i.heff:
                    q.gen_series(leng=_leng)
                    #print(q.series.sum())
                    _heffls = _heffls + q.series
        #print(_effls.sum())
        
        df_sum = pd.DataFrame()
        for i in Set_Point_lst:
            i.gen_dataframe()
            if i.ss_time:
                #print(i.sum_rows)
                df_sum = pd.concat([df_sum, i.sum_rows])
        df_sum = df_sum.sort_values(by=['Init[s]']).reset_index(drop=True)
        display(df_sum)
            
        if SS:        
            _bmask = pd.Series([True if i in _bls else False for i in range(0, _leng)])
            df = df.where(_bmask, 0)
        else:
            pass
    
    if Ploting:
        #plt.clf()
        fig, (ax_TC, ax_DFM, ax_GA, ax_heff) = plt.subplots(4, constrained_layout=True, figsize=(10, 10), sharex=False)
        fig.canvas.toolbar_position = 'left'
        
        df_GA = pd.DataFrame({
            'GA_H2':df['GA_H2'],
            'GA_CO':df['GA_CO'],
            'GA_CO2':df['GA_CO2'],
            'GA_CH4':df['GA_CH4'],
            'GA_N2':df['GA_N2'],
        }, index=df.index)
        df_DFM = pd.DataFrame({
            'DFM_AOG':df['DFM_AOG'],
            'DFM_RichGas':df['DFM_RichGas'],
        }, index=df.index)

        df['TC10'].plot(legend=True, ax=ax_TC, kind='area', color='lightblue')
        df['TC6'].plot(legend=True, ax=ax_TC)
        df['TC7'].plot(legend=True, ax=ax_TC)
        df['TC8'].plot(legend=True, ax=ax_TC)
        df['TC9'].plot(legend=True, ax=ax_TC)
        df['TC11'].plot(legend=True, ax=ax_TC)
        df['EVA_Out'].plot(legend=True, ax=ax_TC,
                           title=f'TC_{Table_name}',
                           xlim=Time, 
                           xticks=range(Time[0],Time[1],1200),
                           ylabel='Temp[oC]',
                           xlabel='Time[s]',
                           ylim=(0,600), 
                           yticks=range(0,600,100),
                          )
        ax_TC_2 = ax_TC.twinx()
        ax_TC_2.set_ylim([0, 600])
        ax_TC_2.set_yticks(range(0,600,100)) 

        df_DFM.plot(legend=True, ax=ax_DFM, kind='area', stacked=True, 
                title=f'DFM_{Table_name}',
                ylabel='Gas Production [LPM]',
                #grid=True,
                xlim=Time, 
                xticks=range(Time[0],Time[1],1200),
                ylim=(0,140), 
                yticks=range(0,140,10)
               )
        ax_DFM_2 = df['Scale'].plot(legend=True, ax=ax_DFM, secondary_y=True, xlabel='Time[s]',)
        ax_DFM_2.set_ylabel('Scale [g/min]')
        ax_DFM_2.set_ylim(0,100)

        df_GA.plot(legend=True, ax=ax_GA, kind='area', stacked=True, 
                title=f'GA_{Table_name}', 
                ylabel='Gas Composition [%]', 
                #grid=True,  
                xlim=Time, 
                xticks=range(Time[0],Time[1],1200),
                ylim=(0,100), 
                yticks=range(0,100,10)
               )
        df['CO_line'] = df['GA_CO']
        ax_GA_2 = df['CO_line'].plot(legend=True, ax=ax_GA, secondary_y=True, xlabel='Time[s]',)
        ax_GA_2.set_ylabel('CO [%]')
        ax_GA_2.set_ylim(0,5)

        _heffls.plot(legend=False, ax=ax_heff, kind='area', stacked=False, 
                title=f'heff_{Table_name}', 
                ylabel='Enthalpy Eff. [%]',
                 xlabel='Time[s]',
                #grid=True,  
                xlim=Time, 
                xticks=range(Time[0],Time[1],1200),
                ylim=(0,100), 
                yticks=range(0,100,10)
               )
        
        plt.show()
    else:
        pass

In [18]:
username = os.environ.get("db_user")
password = os.environ.get("db_pwd")

def update_db_name(change):
    #print(change.new)
    global db_name
    db_name = change.new
    
list_db_name = widgets.RadioButtons(
        options=['reformer', 'catalyst', 'Reformer_BW', 'Reformer_SE'],
        description='Select Database:',
        disabled=False
        )
db_name = list_db_name.value
list_db_name.observe(update_db_name, 'value')
display(list_db_name)

RadioButtons(description='Select Database:', options=('reformer', 'catalyst', 'Reformer_BW', 'Reformer_SE'), v…

In [19]:
try:
    # Connect to MariaDB Platform
    conn = mariadb.connect(
        user=username,
        password=password,
        host="localhost",
        port=3306,
        database=db_name,
        autocommit=True
    )

    # Get Cursor for tx
    cur = conn.cursor(named_tuple=False)
    #print(conn)
    print(cur)
    print(f"Connected to {db_name}@localhost ")
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

Connected to reformer@localhost 


In [20]:
Set_Point_lst = [SerEnergy_Set_Point(name= '18A', temp=300, weight_rate=18.91, dummy=0), 
                 SerEnergy_Set_Point(name= '34A', temp=305, weight_rate=31.85, dummy=0), 
                 SerEnergy_Set_Point(name= '51A', temp=315, weight_rate=49.02, dummy=9.5), 
                 SerEnergy_Set_Point(name= '68A', temp=325, weight_rate=63.1, dummy=0), 
                 SerEnergy_Set_Point(name= '85A', temp=332, weight_rate=81.45, dummy=12.5)]

cur.execute('SHOW tables')
table_dropdown = widgets.Dropdown(options=[u for i in cur.fetchall()[:] for u in i][::-1], description='Table')
cur.execute(f"SELECT * FROM {table_dropdown.value}")
cur.fetchall()
time_slider = widgets.IntRangeSlider(value=[0, cur.rowcount], min=0, max=cur.rowcount, step=1200, description='Time Span', continuous_update=False)
checkbox = widgets.Checkbox(value=False, description='Steady-State Only', disabled=False, indent=False)
toggle_calc = widgets.ToggleButton(value=False, description='Only Calc', disabled=False, button_style='success', tooltip='Description',icon='check')
toggle_plot = widgets.ToggleButton(value=False, description='Plot it!', disabled=False, button_style='info', tooltip='Description',icon='check')

def update_time_range(*args):
    cur.execute(f"SELECT * FROM {table_dropdown.value}")
    cur.fetchall()
    time_slider.max = cur.rowcount
table_dropdown.observe(update_time_range, 'value')

plot = interactive_output(plot, {'Table_name':table_dropdown, 'Time':time_slider, 'SS':checkbox, 'Calc': toggle_calc, 'Ploting':toggle_plot})
toggle = HBox([toggle_calc, toggle_plot])
control = HBox([table_dropdown, time_slider, checkbox])
widgets.VBox([toggle, control, plot])

In [7]:
conn.close()